In [1]:
%autosave 20
import math 
import pandas as pd

Autosaving every 20 seconds


In [8]:
#Load the dataset of Titanic train
df=pd.read_csv("train.csv")
df.head()
df = df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
df.head()
df.columns
xtrain=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']
ytrain=['Survived']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [9]:
df= df.fillna(df.mean())

df1 =[df]

for values in df1:    
    values.loc[ values['Age'] <= 22, 'Age'] = 0
    values.loc[(values['Age'] > 22) & (values['Age'] <= 29), 'Age'] = 1
    values.loc[(values['Age'] > 29) & (values['Age'] <= 35), 'Age'] = 2
    values.loc[(values['Age'] > 35) & (values['Age'] <= 80), 'Age'] = 3
df['Age']=df['Age'].astype(int)

for values in df1:    
    values.loc[ values['Fare'] <= 7.9, 'Fare'] = 0
    values.loc[(values['Fare'] > 7.9) & (values['Fare'] <= 14.45), 'Fare'] = 1
    values.loc[(values['Fare'] > 14.45) & (values['Fare'] <= 30), 'Fare'] = 2
    values.loc[(values['Fare'] > 30) & (values['Fare'] <= 513), 'Fare'] = 3
df['Fare']=df['Fare'].astype(int)

In [10]:
df=df.dropna()
df=df[xtrain+ytrain]
# df.loc[df['play']=='no','play']=0
# df.loc[df['play']=='yes','play']=1
# df.head() 
df=df.dropna()
P,N=df['Survived'].value_counts()
print("yes :",P)
print("No :",N)
df.head()

yes : 549
No : 340


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,0,1,0,0,S,0
1,1,female,3,1,0,3,C,1
2,3,female,1,0,0,1,S,1
3,1,female,2,1,0,3,S,1
4,3,male,2,0,0,1,S,0


In [11]:
# Function to calculate information gain
def informationGain(p,n):
    if(p==0):
        return 0
    elif(n==0):
        return 0    
    IG= -(p/(p+n))*math.log2(p/(p+n)) - (n/(p+n))*math.log2(n/(p+n))
    return round(IG,3)

In [12]:
IG=informationGain(P,N)
print("Total Information Gain : ",IG)

Total Information Gain :  0.96


In [22]:
j=0
Gain = pd.DataFrame(columns=['column', 'gain'])
for col in range(0,len(df.columns)-1):
    b=df.columns[col]
    print(b)
    cmn=[]

    cmn=df.iloc[:,col].unique()
    ig=[]
    p1=[]
    n1=[]
    
    
    for col1 in range(0,len(cmn)):
        p=n=0
        print(cmn[col1])
        s=df.groupby(b).get_group(cmn[col1])
#         print(s)
        p=len(s[s['Survived']==1])
        n=len(s[s['Survived']==0])    
        ig.append(informationGain(p,n))
        p1.append(p)
        n1.append(n)
#         print("information gain of ",cmn[col1], "is ",ig[col1])
             
    E=0.0
    for i in range(0,len(cmn)):
        E=E+(((p1[i]+n1[i])/(P+N))*ig[i])
        
    print("Entropy of ", b, "is ",round(E,3))
    print("Gain of ",b, "is ",round(IG-E,3))
    Gain=Gain.append([{'column':b,'gain':round(IG-E,3)}], ignore_index=True)

#     print(Gain)
    print("***********")  


Sex
male
female
Entropy of  Sex is  0.744
Gain of  Sex is  0.216
***********
Pclass
3
1
2
Entropy of  Pclass is  0.877
Gain of  Pclass is  0.083
***********
Fare
0
3
1
2
Entropy of  Fare is  0.896
Gain of  Fare is  0.064
***********
SibSp
1
0
3
4
2
5
8
Entropy of  SibSp is  0.926
Gain of  SibSp is  0.034
***********
Parch
0
1
2
5
3
4
6
Entropy of  Parch is  0.935
Gain of  Parch is  0.025
***********
Embarked
S
C
Q
Entropy of  Embarked is  0.939
Gain of  Embarked is  0.021
***********
Age
0
3
1
2
Entropy of  Age is  0.958
Gain of  Age is  0.002
***********


In [14]:
print(Gain.sort_values(by=['gain'],ascending=False))

     column   gain
1       Sex  0.216
0    Pclass  0.083
5      Fare  0.064
3     SibSp  0.034
4     Parch  0.025
6  Embarked  0.021
2       Age  0.002


In [17]:
c=Gain.sort_values(by=['gain'],ascending=False).column.tolist()
c.append('Survived')
c

['Sex', 'Pclass', 'Fare', 'SibSp', 'Parch', 'Embarked', 'Age', 'Survived']

In [18]:
df=df[c]
df.head()

,Sex,Pclass,Fare,SibSp,Parch,Embarked,Age,Survived
0,male,3,0,1,0,S,0,0
1,female,1,3,1,0,C,3,1
2,female,3,1,0,0,S,1,1
3,female,1,3,1,0,S,2,1
4,male,3,1,0,0,S,2,0
